In [2]:
import pylidc as pl
import numpy as np
import glob
import os

# 実験

## Textrueが1のスキャンデータをとってくる

In [3]:
type(pl.query(pl.Annotation).filter(pl.Annotation.texture == 1).count())
# => 480枚
len(pl.query(pl.Annotation).filter(pl.Annotation.texture == 1).all())

480

In [4]:
i = 0

In [5]:
# 例えばtextureが0のものの1つ目
ann = pl.query(pl.Annotation).filter(pl.Annotation.texture == 1)[0]

ann

Annotation(id=88,scan_id=13)

In [6]:
scan = pl.query(pl.Scan).filter(pl.Scan.id == ann.scan_id).all()

scan

[Scan(id=13,patient_id=LIDC-IDRI-0002)]

In [7]:
scan[i].cluster_annotations()

[[Annotation(id=88,scan_id=13), Annotation(id=89,scan_id=13)]]

In [8]:
dir_name = scan[i].patient_id # これがディレクトリネームになる

dir_name

'LIDC-IDRI-0002'

In [9]:
# スライス面の番号をとってくる
slices = [np.array([a.centroid for a in group]).mean(0) for group in scan[i].cluster_annotations()]

for j in range(len(slices)):
    slice_num = slices[j][2]
    print (int(slice_num)) # スライス番号

185


In [10]:
# ディレクトリネームとスライス面の番号からdcmファイルを取り出す
path = glob.glob('/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/**/**/*')
path1 = glob.glob('/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-26851/**/*')
path2 = glob.glob('/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/**/*')
path3 = glob.glob('/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/*')
print (len(path))
print (len(path1))
print (len(path2))
print (len(path3))
print (path)
print (path1)
print (path2)
print (path3)

264
2
262
2
['/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-26851/3000972.000000-22902/070.xml', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-26851/3000972.000000-22902/1-1.dcm', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/3000522.000000-04919/071.xml', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/3000522.000000-04919/1-001.dcm', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/3000522.000000-04919/1-002.dcm', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/3000522.000000-04919/1-003.dcm', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/3000522.000000-04919/1-004.dcm', '/Volumes/masashi/workspace/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/01-01-2000-98329/3000522.000000-049

# 完成版

In [11]:
# 指定したpatient_idと欲しいスライス番号を与えれば，それに対するdicomファイルを返す
# スキャンデータとマスクデータを取り出すメソッドを作る
def hogehoge():
    for i in range(pl.query(pl.Annotation).filter(pl.Annotation.texture == 1).count()):
        # textureが1のものの全てのアノテーションデータ
        ann = pl.query(pl.Annotation).filter(pl.Annotation.texture == 1)[i]
        # アノテーションに対する全てのスキャンデータ
        scan_list = (pl.query(pl.Scan).filter(pl.Scan.id == ann.scan_id)).all()
        # 全てのスキャンデータからディレクトリネームとスライス番号を取り出す
        for a_scan in scan_list:
            dir_name = a_scan.patient_id # ディレクトリネーム
            slices = [np.array([a.centroid for a in group]).mean(0) for group in a_scan.cluster_annotations()]
            
            # スライス全てを取り出してアノテーションされているスキャンデータを取り出す
            for j in range(len(slices)):
                slice_num = slices[j][2]
                print (int(slice_num)) # スライス番号になる
                

In [12]:
# hogehoge()

In [14]:
# 任意のtextureの値のアノテーションデータを全て取り出す
def get_texture(texture_val):
    return (pl.query(pl.Annotation).filter(pl.Annotation.texture <= texture_val)).all()

# アノテーションデータに対応するスキャンデータを返す
def get_scan(annotation):
    return (pl.query(pl.Scan).filter(pl.Scan.id == annotation.scan_id)).all()

# スキャンデータのpatient_idを返す
def get_patient_id(scan):
    return scan.patient_id

# アノテーションされたスキャンデータからアノテーションされているスライス面のスライス番号を返す
def get_slices(scan):
    slices =  [np.array([a.centroid for a in group]).mean(0) for group in scan.cluster_annotations()]
    slice_list = []
    for a in range(len(slices)):
        a_slice = int(slices[a][2])
        slice_list.append(a_slice)
    return slice_list

# アノテーションされているスキャンデータのdicomを取り出す
def get_path(dir_name):
    init_path = '/Volumes/masashi/workspce/0_KML/0_medical/0_data/LIDC-IDRI/'
    print (init_path + dir_name + '/*')
    if len(glob.glob(init_path + dir_name + '/*')) >= 2:
        return '1以上'
    else:
        return '1'

In [13]:
ann_index = 0
scan_index = 0
for i, ann_index in enumerate(range(1012)):
    ann = get_texture(1)
    print ('ann', ann[ann_index])
    scan = get_scan(ann[ann_index])
    print ('scan', scan)
    patient_id = get_patient_id(scan[scan_index])
    print ('patient_id', patient_id) # ディレクトリネーム
    slices = get_slices(scan[scan_index])
    print ('slices', slices)
    path = get_path(patient_id)
    print (i+1, ':', path)
    print ('----------------------------------')

ann Annotation(id=88,scan_id=13)
scan [Scan(id=13,patient_id=LIDC-IDRI-0002)]
patient_id LIDC-IDRI-0002
slices [185]
/Volumes/masashi/workspce/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0002/*
1 : 1
----------------------------------
ann Annotation(id=102,scan_id=14)
scan [Scan(id=14,patient_id=LIDC-IDRI-0003)]
patient_id LIDC-IDRI-0003
slices [66, 75, 82, 83]
/Volumes/masashi/workspce/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0003/*
2 : 1
----------------------------------
ann Annotation(id=123,scan_id=17)
scan [Scan(id=17,patient_id=LIDC-IDRI-0006)]
patient_id LIDC-IDRI-0006
slices [54, 68, 70, 83]
/Volumes/masashi/workspce/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0006/*
3 : 1
----------------------------------
ann Annotation(id=131,scan_id=19)
scan [Scan(id=19,patient_id=LIDC-IDRI-0008)]
patient_id LIDC-IDRI-0008
slices [70, 88]
/Volumes/masashi/workspce/0_KML/0_medical/0_data/LIDC-IDRI/LIDC-IDRI-0008/*
4 : 1
----------------------------------
ann Annotation(id=237,scan_id=27)
sc

KeyboardInterrupt: 